In [1]:
import os
import sys
import torch
import gpytorch
from tqdm.auto import tqdm
import timeit
import numpy as np
import pandas as pd
if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

In [2]:
dpath = 'snelson.csv'
snel = pd.read_csv(dpath)
snel = snel.iloc[:200]#np.random.permutation(range(snel.shape[0]))[:100]]
snel_x = torch.from_numpy(snel.x.to_numpy()).unsqueeze(-1).float()
snel_y = torch.from_numpy(snel.y.to_numpy()).float()

In [3]:
from bi_gp.gaussian_matrix import latticefilter

In [4]:
def rel_err(x,y):
    return (x-y).abs().mean()/(x.abs().mean()+y.abs().mean())

In [5]:
snel_y.shape

torch.Size([200])

In [6]:
X = snel_x
f = snel_y[:,None]
#out = torch.zeros(1000000,16)
#print(lattice.testfunc(torch.ones((2,3))))
#src[:,-1] = torch.ones(1000)
# print(X)
# print(f)


In [7]:
Kxx = torch.exp(-((X[None]-X[:,None])**2).sum(-1)/2)#/(2*np.pi)**(X.shape[-1]/2)

In [8]:
mvm_gt = Kxx@f
mvm_lattice = latticefilter(f,X,3)

In [9]:
#mvm_gt[:,0]

In [10]:
#mvm_lattice[:,0]

In [11]:
#mvm_gt/mvm_lattice

In [12]:
rel_err(mvm_gt,mvm_lattice/(mvm_lattice/mvm_gt).mean())

tensor(0.0387)

In [13]:
print(latticefilter(src,ref,1))

NameError: name 'src' is not defined

In [14]:
from gpytorch.kernels.matern_kernel import matern

ImportError: cannot import name 'matern' from 'gpytorch.kernels.matern_kernel' (/home/marc/anaconda3/envs/default/lib/python3.7/site-packages/gpytorch/kernels/matern_kernel.py)

In [15]:
from gpytorch.kernels import MaternKernel

In [16]:
K = MaternKernel(nu=1.5)

In [17]:
x = torch.linspace(0,4,3).float()[:-1,None]

In [18]:
K(x,x).evaluate()[0]

tensor([1.0000, 0.0405], grad_fn=<SelectBackward>)

In [19]:
from gpytorch.kernels import RBFKernel

In [20]:
K = RBFKernel()
x = torch.linspace(0,2,3).float()[:-1,None]
K(x,x).evaluate()[0]

tensor([1.0000, 0.3532], grad_fn=<SelectBackward>)

In [24]:
for i in range(1,6):
    x = torch.arange(6).float()[:,None]/np.sqrt(i/2)
    print(K(x,x).evaluate()[0].data)

tensor([1.0000e+00, 1.2476e-01, 2.4227e-04, 7.3224e-09, 3.4448e-15, 2.5225e-23])
tensor([1.0000e+00, 3.5321e-01, 1.5565e-02, 8.5571e-05, 5.8693e-08, 5.0224e-12])
tensor([1.0000e+00, 4.9968e-01, 6.2340e-02, 1.9419e-03, 1.5103e-05, 2.9328e-08])
tensor([1.0000e+00, 5.9432e-01, 1.2476e-01, 9.2505e-03, 2.4227e-04, 2.2411e-06])
tensor([1.0000e+00, 6.5950e-01, 1.8917e-01, 2.3601e-02, 1.2807e-03, 3.0225e-05])
